In [14]:
%load_ext kedro.extras.extensions.ipython

The kedro.extras.extensions.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.extras.extensions.ipython


In [15]:
%reload_kedro

[03/04/24 17:35:44] INFO     Resolved project path as: /home/sossa/raftel.                          ]8;id=312063;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=174365;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[03/04/24 17:35:45] INFO     Kedro project raftel                                                   ]8;id=831256;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=424642;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=650707;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=155153;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#104\104]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=844107;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=986608;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/ipython/__init__.py#110\110]8;;\

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [17]:
spark = SparkSession.builder.getOrCreate()

In [18]:
import pandas as pd
import numpy as np

# Define the file names
file_name = "House_A"

# Create an empty DataFrame to store the data
columns = [
    "ID",
    "ACTIVITY"
]

df = pd.DataFrame(columns=columns)

# Load data from each file and append to the DataFrame
file_path = "/home/sossa/raftel/data/01_raw/unloaded/Aras/Labels/{}.csv".format(file_name)  # assuming the files have a .txt extension
df = pd.read_csv(file_path, sep=',', header=0, names=columns)

print(df)

    ID              ACTIVITY
0    1                 Other
1    2             Going Out
2    3   Preparing Breakfast
3    4      Having Breakfast
4    5       Preparing Lunch
5    6          Having Lunch
6    7      Preparing Dinner
7    8         Having Dinner
8    9        Washing Dishes
9   10          Having Snack
10  11              Sleeping
11  12           Watching TV
12  13              Studying
13  14         Having Shower
14  15             Toileting
15  16               Napping
16  17        Using Internet
17  18          Reading Book
18  19               Laundry
19  20               Shaving
20  21        Brushing Teeth
21  22  Talking on the Phone
22  23    Listening to Music
23  24              Cleaning
24  25   Having Conversation
25  26          Having Guest
26  27      Changing Clothes


In [19]:
df_spark = spark.createDataFrame(df)

In [20]:
df_spark.printSchema()

root
 |-- ID: long (nullable = true)
 |-- ACTIVITY: string (nullable = true)



In [21]:
catalog.save("aras_a_labels@spark", df_spark)

[03/04/24 17:36:05] INFO     Saving data to 'aras_a_labels@spark' (SparkDataSet)...             ]8;id=255751;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=427013;file:///home/sossa/miniconda3/envs/raftel/lib/python3.8/site-packages/kedro/io/data_catalog.py#382\382]8;;\

In [ ]:
df_spark.count()

In [ ]:
labels = df_spark.select("LR1", "LR2").distinct().withColumn("LABEL", F.monotonically_increasing_id())

In [ ]:
df_spark = df_spark.join(
    labels,
    ["LR1", "LR2"],
    "LEFT"
).orderBy(
    "SEC"
)

In [ ]:
df_spark.write.partitionBy("DAY").mode("append").format("delta").option("overwriteSchema", "true").save("/home/sossa/raftel/data/01_raw/aras_a")